## Authorization to the DEDL services

In [1]:
from dedllogin import DEDL_auth
import requests
import json
import os
from getpass import getpass
from IPython.display import JSON

username = input("Please input your DESP username or email: ")
password = getpass("Please input your DESP password: ")

auth = DEDL_auth(username, password)

access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

Please input your DESP username or email:  jesper.dejby@ri.se
Please input your DESP password:  ········


DEDL/DESP Access Token Obtained Successfully


## Download data

We are aiming to download the Fraction of Absorbed Photosynthetically Active Radiation dataset described here:  https://hda.data.destination-earth.eu/ui/dataset/EO.CLMS.DAT.GLO.FAPAR300_V1

In [2]:
response_irr = requests.post("https://hda.data.destination-earth.eu/stac/search", headers=auth_headers, json={
    "collections": ["EO.CLMS.DAT.GLO.FAPAR300_V1"],
    "datetime": "2024-05-31T00:00:00Z/2024-06-01T00:00:00Z",
    'bbox': [10,53,30,66]
})

In [3]:
response_irr.json()["features"][0]["assets"]["downloadLink"]["href"]

'https://hda.data.destination-earth.eu/stac/collections/EO.CLMS.DAT.GLO.FAPAR300_V1/items/c_gls_FAPAR300-RT0_202406100000_GLOBE_OLCI_V1.1.2/download?provider=wekeo'

In [4]:
from IPython.display import JSON

product_irr = []
assets = []
for i in response_irr.json()["features"]:
    # Download NC
    product_irr.append(i["assets"]["downloadLink"]["alternate"]["origin"]["href"])
    # Download using main link
    #product_irr.append(i["assets"]["downloadLink"]["href"])
    assets.append(i["id"])

In [5]:
JSON(response_irr.json())

<IPython.core.display.JSON object>

In [6]:
from tqdm import tqdm
import time

# Define a list of assets to download
download_url = product_irr[0]
filename = assets[0]
response = requests.get(download_url, headers=auth_headers)
total_size = int(response.headers.get("content-length", 0))

print(f"downloading {filename}")

with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
    with open(filename, 'wb') as f:
        for data in response.iter_content(1024):
            progress_bar.update(len(data))
            f.write(data)

downloading c_gls_FAPAR300-RT0_202406100000_GLOBE_OLCI_V1.1.2


100%|██████████| 2.19G/2.19G [00:06<00:00, 351MB/s] 


## Read data

We read the data and subset it too the baltic sea region.

In [7]:
import rioxarray
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import netCDF4 as nc

In [8]:
ds = xr.open_dataset("c_gls_FAPAR300-RT0_202406100000_GLOBE_OLCI_V1.1.2")

In [9]:
lat_bnds, lon_bnds = [66, 53], [10, 30]
baltic_sea_subset=ds.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

In [14]:
baltic_sea_subset.variables["FAPAR"]

<xarray.Variable (time: 1, lat: 4368, lon: 6720)>
array([[[  nan,   nan, ..., 0.464, 0.532],
        [  nan,   nan, ..., 0.62 , 0.612],
        ...,
        [  nan,   nan, ..., 0.656, 0.468],
        [0.808, 0.868, ..., 0.408, 0.504]]], dtype=float32)
Attributes:
    standard_name:  fraction_of_surface_downwelling_photosynthetic_radiative_...
    long_name:      Fraction of Absorbed Photosynthetically Active Radiation ...
    grid_mapping:   crs
    units:          
    valid_range:    [  0 235]

In [15]:
print(baltic_sea_subset.variables["FAPAR"].max(), baltic_sea_subset.variables["FAPAR"].min())

<xarray.Variable ()>
array(0.94000006, dtype=float32) <xarray.Variable ()>
array(0., dtype=float32)


## Conclusion

We have a requirement for the Irradiation to be measured in microenstein per m2. In addition range of the data should not be between 0 and 1, but rather somewhere between 1000 and 2000 depending on the season. Hence we can't use this dataset as it seems to be normalised already and we have no way of verifying the values.